# Vector Addition with Triton

In [17]:
import torch
import triton
import triton.language as tl

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
@triton.jit
def add_kernel(x_ptr,  # *Pointer* to first input vector.
               y_ptr,  # *Pointer* to second input vector.
               output_ptr,  # *Pointer* to output vector.
               n_elements,  # Size of the vector.
               BLOCK_SIZE: tl.constexpr,  # Number of elements each program should process.
               # NOTE: `constexpr` so it can be used as a shape value.
               ):
    # There are multiple 'programs' processing different data. We identify which program
    # we are here:
    pid = tl.program_id(axis=0)  # We use a 1D launch grid so axis is 0.
    # This program will process inputs that are offset from the initial data.
    # For instance, if you had a vector of length 256 and block_size of 64, the programs
    # would each access the elements [0:64, 64:128, 128:192, 192:256].
    # Note that offsets is a list of pointers:
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    # Create a mask to guard memory operations against out-of-bounds accesses.
    mask = offsets < n_elements
    # Load x and y from DRAM, masking out any extra elements in case the input is not a
    # multiple of the block size.
    x = tl.load(x_ptr + offsets, mask=mask)
    y = tl.load(y_ptr + offsets, mask=mask)
    output = x + y
    # Write x + y back to DRAM.
    tl.store(output_ptr + offsets, output, mask=mask)

In [19]:
def add(x: torch.Tensor, y: torch.Tensor):
    # We need to preallocate the output.
    output = torch.empty_like(x)
    assert x.device == DEVICE and y.device == DEVICE and output.device == DEVICE
    n_elements = output.numel()
    # The SPMD launch grid denotes the number of kernel instances that run in parallel.
    # It is analogous to CUDA launch grids. It can be either Tuple[int], or Callable(metaparameters) -> Tuple[int].
    # In this case, we use a 1D grid where the size is the number of blocks:
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )
    # NOTE:
    #  - Each torch.tensor object is implicitly converted into a pointer to its first element.
    #  - `triton.jit`'ed functions can be indexed with a launch grid to obtain a callable GPU kernel.
    #  - Don't forget to pass meta-parameters as keywords arguments.
    add_kernel[grid](x, y, output, n_elements, BLOCK_SIZE=1024)
    # We return a handle to z but, since `torch.cuda.synchronize()` hasn't been called, the kernel is still
    # running asynchronously at this point.
    return output

To explain a bit more what this line
`grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )`
is doing, imagine that in our case each program (analogous to a CUDA block) will handle a chunk of 1024 elements (analogous to threads within a block in CUDA). We also know that each vector we are adding is of size 98432. We want to divide this vector into chunks for the GPU to process in parallel. The GPU needs to know how many such programs (or blocks, in CUDA terms) we should have to lay out the grid. So what we did is divide the size of the vector by the size of each program's chunk, i.e., `n_elements / BLOCK_SIZE`.

As for why we need to do a ceiling division, that’s because if we take
`98432 / 1024 = 96.125`
obviously that doesn’t make sense as we can't launch 96.something programs. And even if we floor it to 96, the total number of elements processed would be
`96 * 1024 = 98304`
which means we’d be missing 128 elements.

By doing a ceiling division, we launch 97 programs instead of 96. The first 96 programs will process 1024 elements each, and the last one will handle the remaining 128 elements.

Actually, a bit of a caveat: the last program will still try to process 1024 elements, because that’s how each program is instructed to operate. So this last one might go past the end of the vector. That’s why the kernel in the `add_kernel` code above uses a mask to prevent out-of-bounds access, by doing:
`mask = offsets < n_elements`

-------------

As for the following line, to understand what’s happening here
`add_kernel[grid](x, y, output, n_elements, BLOCK_SIZE=1024)`
we need to first talk about what `@triton.jit` does. When we write a function and decorate it with `@triton.jit`, we are telling Triton to compile that function into a GPU kernel. That means it no longer behaves like a normal Python function and we can’t just call it like `add_kernel(...)`. Instead, we launch it using the syntax `add_kernel...`, which is analogous to CUDA’s `<<<grid, block>>>` launch syntax.

In this syntax, the part inside the square brackets, `[grid]`, specifies the launch configuration, i.e., how many parallel instances of the kernel to run. The part inside the parentheses, `(x, y, output, n_elements, BLOCK_SIZE=1024)`, contains the actual arguments passed into the kernel. This includes normal runtime inputs like `x`, `y`, and `n_elements`, and also meta-parameters like `BLOCK_SIZE`.

A meta-parameter is something known at compile time, not runtime. In Triton, meta-parameters are defined inside the kernel with `tl.constexpr`, like in `BLOCK_SIZE: tl.constexpr`. These values must be passed as keyword arguments when launching the kernel, e.g., `BLOCK_SIZE=1024`, and Triton will use them to generate optimised GPU code. Even though `BLOCK_SIZE` looks like a normal argument in the function definition, Triton treats it specially.

Now coming to `grid`, earlier in the code we defined it like this:
`grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )`
This `grid` is a lambda function that takes in a dictionary called `meta`, and returns a tuple — in this case, the number of programs to launch. What’s important is that we never call `grid(meta)` ourselves. Instead, Triton automatically constructs a `meta` dictionary using the keyword arguments we provide — in this case, it builds `meta = {'BLOCK_SIZE': 1024}`.

When we write `add_kernelgrid`, Triton internally does something like:
`grid({'BLOCK_SIZE': 1024})`
which gives us
`(triton.cdiv(n_elements, 1024), )`
This result is used as the actual launch configuration — for example, launching 97 programs if `n_elements = 98432` and `BLOCK_SIZE = 1024`.

In [20]:
torch.manual_seed(0)
size = 98432
x = torch.rand(size, device=DEVICE)
y = torch.rand(size, device=DEVICE)
output_torch = x + y
output_triton = add(x, y)
print(output_torch)
print(output_triton)
print(f'The maximum difference between torch and triton is '
      f'{torch.max(torch.abs(output_torch - output_triton))}')

tensor([1.3713, 1.3076, 0.4940,  ..., 0.6724, 1.2141, 0.9733], device='cuda:0')
tensor([1.3713, 1.3076, 0.4940,  ..., 0.6724, 1.2141, 0.9733], device='cuda:0')
The maximum difference between torch and triton is 0.0
